In [1]:
!pip install -q transformers datasets librosa soundfile torch accelerate evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
import torch
import librosa
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from google.colab import files
import io
import soundfile as sf
import numpy as np
import evaluate

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

print(f"Using device: {device}")
print(f"Using dtype: {torch_dtype}")

wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")
print("WER and CER metrics loaded.")

Using device: cpu
Using dtype: torch.float32


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WER and CER metrics loaded.


In [4]:
from huggingface_hub import login

login()

In [5]:
whisper_model_id = "iTzMiNOS/whisper-small-finetuned-new"

print(f"Loading Whisper model: {whisper_model_id}")
whisper_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    whisper_model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)


whisper_model.to(device)

whisper_processor = AutoProcessor.from_pretrained(whisper_model_id)

pipe_transcribe = pipeline(
    "automatic-speech-recognition",
    model=whisper_model,
    tokenizer=whisper_processor.tokenizer,
    feature_extractor=whisper_processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=False,
    torch_dtype=torch_dtype,
    device=device,
)
print("Whisper model loaded.")

Loading Whisper model: iTzMiNOS/whisper-small-finetuned-new


config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/356 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Device set to use cpu


Whisper model loaded.


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/automatic_speech_recognition.py:312: FutureWarning: `max_new_tokens` is deprecated and will be removed in version 4.49 of Transformers. To remove this warning, pass `max_new_tokens` as a key inside `generate_kwargs` instead.
  warnings.warn(


In [9]:
t5_model_id = "iTzMiNOS/t5-small-2-4-ipo-txtonly"

print(f"Loading T5 model: {t5_model_id} for Text-to-JSON conversion")
try:
    t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_id)
    t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_model_id)
    t5_model.to(device)
    print("T5 model loaded successfully.")
except Exception as e:
    print(f"ERROR: Failed to load T5 model '{t5_model_id}'.")
    print(f"Please check the model ID and ensure it's accessible on Hugging Face Hub.")
    print(f"Error details: {e}")
    t5_model = None
    t5_tokenizer = None

Loading T5 model: iTzMiNOS/t5-small-2-4-ipo-txtonly for Text-to-JSON conversion
T5 model loaded successfully.


In [12]:
def process_audio_whisper_t5_with_metrics(audio_data, sampling_rate, reference_text, max_length=256, min_length=5):
    """
    Transcribes audio using Whisper, calculates WER/CER against reference text,
    and then processes the transcription with the loaded T5 model (expected Text-to-JSON).

    Args:
        audio_data (np.array): The audio waveform.
        sampling_rate (int): The sampling rate of the audio.
        reference_text (str): The ground truth transcription.
        max_length (int): Max tokens for T5 generation (JSON output).
        min_length (int): Min tokens for T5 generation (JSON output).

    Returns:
        tuple: (transcription, t5_output, wer_score, cer_score)
               Returns (None, None, None, None) if transcription fails.
               Returns (transcription, None, wer_score, cer_score) if T5 fails or wasn't loaded.
    """
    print("1. Starting transcription...")
    whisper_input = {"raw": audio_data, "sampling_rate": sampling_rate}
    transcription_result = pipe_transcribe(whisper_input)
    hypothesis_text = transcription_result["text"].strip()

    if not hypothesis_text:
        print("Transcription failed or produced empty text.")
        return None, None, None, None

    print(f"2. Transcription complete (Hypothesis): '{hypothesis_text}'")
    print(f"   Reference:                        '{reference_text}'")

    print("3. Calculating WER and CER...")
    wer_score = None
    cer_score = None
    try:
        wer_score = wer_metric.compute(predictions=[hypothesis_text], references=[reference_text])
        cer_score = cer_metric.compute(predictions=[hypothesis_text], references=[reference_text])
        print(f"   WER: {wer_score:.4f}")
        print(f"   CER: {cer_score:.4f}")
    except Exception as e:
        print(f"   Could not compute metrics: {e}")


    print("4. Starting T5 processing (Text-to-JSON)...")
    t5_output = None

    if t5_model is None or t5_tokenizer is None:
        print("   Skipping T5 processing: Model or Tokenizer not loaded.")
    else:
        try:
            t5_input_text = "translate to json: " + hypothesis_text

            inputs = t5_tokenizer(t5_input_text, return_tensors="pt", max_length=512, truncation=True).to(device)

            output_ids = t5_model.generate(
                inputs['input_ids'],
                num_beams=4,
                max_length=max_length,
                min_length=min_length,
                early_stopping=True
            )
            t5_output = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)
            print(f"5. T5 processing complete.")
        except Exception as e:
            print(f"   T5 processing failed: {e}")
            import traceback
            traceback.print_exc()
            t5_output = None


    return hypothesis_text, t5_output, wer_score, cer_score

In [13]:
print("Please upload an audio file (e.g., wav, mp3, flac):")
uploaded = files.upload()

if not uploaded:
    print("No file uploaded.")
else:
    filename = next(iter(uploaded))
    print(f"\nProcessing uploaded file: {filename}")

    print("\nPlease paste the EXACT ground truth (reference) transcription for the audio below and press Enter:")
    reference_text = input()
    reference_text = reference_text.strip()

    if not reference_text:
        print("No reference text provided. Cannot calculate metrics.")
    else:
        audio_bytes = uploaded[filename]

        try:
            audio_data, sampling_rate = sf.read(io.BytesIO(audio_bytes))
            print(f"Original sampling rate: {sampling_rate}")

            if sampling_rate != 16000:
                print("Resampling audio to 16kHz for Whisper...")
                if audio_data.dtype != np.float32 and audio_data.dtype != np.float64:
                     audio_data = audio_data.astype(np.float32) / np.iinfo(audio_data.dtype).max
                if len(audio_data.shape) > 1 and audio_data.shape[1] > 1:
                    audio_data = audio_data.mean(axis=1)
                if len(audio_data.shape) > 1:
                    audio_data = audio_data.flatten()
                audio_data = librosa.resample(y=audio_data, orig_sr=sampling_rate, target_sr=16000)
                sampling_rate = 16000
            elif len(audio_data.shape) > 1 and audio_data.shape[1] > 1:
                 print("Converting audio to mono...")
                 audio_data = audio_data.mean(axis=1)

            transcription, t5_result, wer, cer = process_audio_whisper_t5_with_metrics(
                audio_data,
                sampling_rate,
                reference_text,
                max_length=256,
                min_length=10
            )

            if transcription is not None:
                print("\n--- Results ---")
                print(f"Reference Text:\n{reference_text}")
                print(f"\nHypothesis Text (Whisper Transcription):\n{transcription}")
                if wer is not None and cer is not None:
                   print(f"\nASR Metrics:\n  WER: {wer:.4f}\n  CER: {cer:.4f}")
                else:
                   print("\nASR Metrics: Could not be calculated.")
                print(f"\nT5 Output (JSON):\n{t5_result if t5_result is not None else 'T5 processing failed, skipped, or model not loaded.'}")
                print("---------------")

                if t5_result:
                    import json
                    try:
                        parsed_json = json.loads(t5_result)
                        print("\nJSON Output successfully parsed.")
                    except json.JSONDecodeError as json_e:
                        print(f"\nWarning: T5 output could not be parsed as valid JSON: {json_e}")

            else:
                print("\nPipeline failed during transcription.")

        except Exception as e:
            import traceback
            print(f"\n--- ERROR ---")
            print(f"An error occurred during audio processing or pipeline execution:")
            print(f"{type(e).__name__}: {e}")
            print("\nTraceback:")
            traceback.print_exc()
            print("-------------")
            print("Please ensure the uploaded file is a valid audio format and the reference text was provided correctly.")

Please upload an audio file (e.g., wav, mp3, flac):


Saving audio.wav to audio (2).wav

Processing uploaded file: audio (2).wav

Please paste the EXACT ground truth (reference) transcription for the audio below and press Enter:
I'll have a Cream of Mushroom with truffle oil and chopped parsley, a Greek salad without protein, add grilled chicken and falafel, a vegan burger with mustard, ketchup, no cheese, vegan cheese, and vegan mayo, plus extra cheese on the side.
Original sampling rate: 24000
Resampling audio to 16kHz for Whisper...
1. Starting transcription...
2. Transcription complete (Hypothesis): 'I'll have a Cream of Mushroom with truffle oil and chopped parsley, a Greek Salad without protein, add grilled chicken and falafel, a Vegan Burger with mustard, ketchup, no cheese, vegan cheese, and vegan mayo, plus extra cheese on the side.'
   Reference:                        'I'll have a Cream of Mushroom with truffle oil and chopped parsley, a Greek salad without protein, add grilled chicken and falafel, a vegan burger with mustard, 